In [1]:
import os
import tensorflow as tf

2025-07-19 21:58:20.377408: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-19 21:58:20.387513: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-19 21:58:20.425011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752942500.479595   22607 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752942500.494879   22607 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-19 21:58:20.542145: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
os.chdir('../')
%pwd

'/home/swapnil-nagar/Documents/ML Projects/Kidney-Disease-Classifier'

In [17]:
os.environ['MLFLOW_TRACKING_UI']='https://dagshub.com/Swapnil5101/Kidney-Disease-Classifier.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='f4b146a8163da584e740a5b5b884330b40ade30b' 

In [ ]:
# model = tf.keras.models.load_model("artifacts/training/model.h5", compile=False)
# model.compile(
#     optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#     metrics=['accuracy']
# )

W0000 00:00:1752176801.974809  110984 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [5]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories, save_json

In [ ]:
class ConfigurationManager:
    def __init__(
        self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Swapnil5101/Kidney-Disease-Classifier.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        
        return eval_config

In [7]:
import dagshub
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [20]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            class_mode="sparse",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        model = tf.keras.models.load_model(path, compile=False)
        model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
        )
        
        return model
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()
        
    def save_score(self):
        scores = {
            "loss": self.score[0],
            "accuracy": self.score[1]
        }
        save_json(path=Path("scores.json"), data=scores)
        
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        # mlflow.set_experiment("Kidney-Disease-Classifier_VGG16")
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        dagshub.init(repo_owner='Swapnil5101', repo_name='Kidney-Disease-Classifier', mlflow=True)
        
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
             
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    model=self.model,
                    artifact_path="model",
                    registered_model_name="Kidney-Disease-Classifier_VGG16"
                )
            else:
                mlflow.keras.log_model(
                    model=self.model,
                    artifact_path="model"
                )
        

In [21]:
try:
    config_manager = ConfigurationManager()
    evaluation_config = config_manager.get_evaluation_config()
    evaluation = Evaluation(config=evaluation_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-07-19 22:41:52,249: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-19 22:41:52,253: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-19 22:41:52,255: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.


/home/swapnil-nagar/Documents/ML Projects/Kidney-Disease-Classifier/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 42s 5s/step - accuracy: 0.7891 - loss: 13.2561
[2025-07-19 22:42:35,083: INFO: common: json file saved at: scores.json]
[2025-07-19 22:42:36,381: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/Swapnil5101/Kidney-Disease-Classifier "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "Swapnil5101/Kidney-Disease-Classifier"

[2025-07-19 22:42:36,390: INFO: helpers: Initialized MLflow to track repo "Swapnil5101/Kidney-Disease-Classifier"]


Repository Swapnil5101/Kidney-Disease-Classifier initialized!

[2025-07-19 22:42:36,392: INFO: helpers: Repository Swapnil5101/Kidney-Disease-Classifier initialized!]
[2025-07-19 22:42:36,451: WARNING: connectionpool: Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Swapnil5101/Kidney-Disease-Classifier.mlflow/api/2.0/mlflow/runs/create]


2025/07/19 22:42:38 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/07/19 22:42:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Kidney-Disease-Classifier_VGG16'.
2025/07/19 22:51:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Kidney-Disease-Classifier_VGG16, version 1
Created version '1' of model 'Kidney-Disease-Classifier_VGG16'.


🏃 View run exultant-lamb-216 at: https://dagshub.com/Swapnil5101/Kidney-Disease-Classifier.mlflow/#/experiments/0/runs/4def7d9a445e469b80dfdbdc4b9a0591
🧪 View experiment at: https://dagshub.com/Swapnil5101/Kidney-Disease-Classifier.mlflow/#/experiments/0


In [ ]:
# import dagshub
# dagshub.init(repo_owner='Swapnil5101', repo_name='Kidney-Disease-Classifier', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)